In [1]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer

In [2]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open(r'intents.json').read())

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']


In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

In [4]:

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [5]:
training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

In [7]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [8]:

hist = model.fit(np.array(trainX), np.array(trainY), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print('Done')


Epoch 1/200
12/12 [==============================] - 1s 5ms/step - loss: 2.9430 - accuracy: 0.0357
Epoch 2/200
12/12 [==============================] - 0s 4ms/step - loss: 2.9345 - accuracy: 0.1071
Epoch 3/200
12/12 [==============================] - 0s 3ms/step - loss: 2.8482 - accuracy: 0.1071
Epoch 4/200
12/12 [==============================] - 0s 4ms/step - loss: 2.7929 - accuracy: 0.1071
Epoch 5/200
12/12 [==============================] - 0s 4ms/step - loss: 2.7114 - accuracy: 0.1786
Epoch 6/200
12/12 [==============================] - 0s 3ms/step - loss: 2.6407 - accuracy: 0.2321
Epoch 7/200
12/12 [==============================] - 0s 4ms/step - loss: 2.6446 - accuracy: 0.2679
Epoch 8/200
12/12 [==============================] - 0s 3ms/step - loss: 2.4913 - accuracy: 0.2857
Epoch 9/200
12/12 [==============================] - 0s 4ms/step - loss: 2.4933 - accuracy: 0.2500
Epoch 10/200
12/12 [==============================] - 0s 5ms/step - loss: 2.3828 - accuracy: 0.3571
Epoch 11/

C:\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
